In [169]:
import math
import time
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC

# Testing how to split correctly the data

In [17]:
df_meta = pd.read_csv('../datasets/raw/HT_Sensor_metadata.dat', delimiter='\t+')
df_meta.shape

(100, 5)

In [77]:
len_indices = df_meta.shape[0]
test_perc = 0.3
n_test = math.floor(test_perc * len_indices)
test_indices = np.random.choice(np.arange(100), size=n_test, replace=False)
print(test_indices)

[58 22 94 39 89 50 35 88  0 73 65 29 62 43 27  4 41 23 98 47 75 36 18 26
 78 10 96 25 13 51]


In [108]:
df_db = pd.read_csv('../datasets/raw/HT_Sensor_dataset.dat', delimiter='\s+')
df_db.shape

(928991, 12)

In [109]:
df_db.set_index('id', inplace=True)
df_db = df_db.join(df_meta, how='inner')
df_db['time'] += df_db['t0']
df_db.set_index(np.arange(df_db.shape[0]), inplace=True)
df_db.head()

,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity,id,date,class,t0,dt
0,12.490250,12.8621,10.3683,10.4383,11.6699,13.4931,13.3423,8.04169,8.73901,26.2257,59.0528,0,07-04-15,banana,13.49,1.64
1,12.490528,12.8617,10.3682,10.4375,11.6697,13.4927,13.3412,8.04133,8.73908,26.2308,59.0299,0,07-04-15,banana,13.49,1.64
2,12.490806,12.8607,10.3686,10.4370,11.6696,13.4924,13.3405,8.04101,8.73915,26.2365,59.0093,0,07-04-15,banana,13.49,1.64
3,12.491084,12.8602,10.3686,10.4370,11.6697,13.4921,13.3398,8.04086,8.73936,26.2416,58.9905,0,07-04-15,banana,13.49,1.64
4,12.491373,12.8595,10.3688,10.4374,11.6699,13.4919,13.3390,8.04087,8.73986,26.2462,58.9736,0,07-04-15,banana,13.49,1.64


In [132]:
bool_list = []
for item in df_db.id:
    if item in test_indices:
        bool_list.append(True)
    else:
        bool_list.append(False)

print(len(bool_list))
bool_list = np.asarray(bool_list)

928991


In [133]:
df_test = df_db[bool_list]
df_test.head()

,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity,id,date,class,t0,dt
0,12.490250,12.8621,10.3683,10.4383,11.6699,13.4931,13.3423,8.04169,8.73901,26.2257,59.0528,0,07-04-15,banana,13.49,1.64
1,12.490528,12.8617,10.3682,10.4375,11.6697,13.4927,13.3412,8.04133,8.73908,26.2308,59.0299,0,07-04-15,banana,13.49,1.64
2,12.490806,12.8607,10.3686,10.4370,11.6696,13.4924,13.3405,8.04101,8.73915,26.2365,59.0093,0,07-04-15,banana,13.49,1.64
3,12.491084,12.8602,10.3686,10.4370,11.6697,13.4921,13.3398,8.04086,8.73936,26.2416,58.9905,0,07-04-15,banana,13.49,1.64
4,12.491373,12.8595,10.3688,10.4374,11.6699,13.4919,13.3390,8.04087,8.73986,26.2462,58.9736,0,07-04-15,banana,13.49,1.64


In [136]:
df_train = df_db[~bool_list]
df_train.head()

,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity,id,date,class,t0,dt
12815,18.610202,12.9677,10.3815,10.4593,11.6892,12.3805,14.1287,7.18476,7.77536,25.7545,58.9859,1,07-05-15,wine,19.61,0.54
12816,18.610491,12.9671,10.3814,10.4590,11.6886,12.3812,14.1280,7.18530,7.77512,25.7560,58.9812,1,07-05-15,wine,19.61,0.54
12817,18.610769,12.9668,10.3813,10.4592,11.6881,12.3820,14.1274,7.18579,7.77504,25.7573,58.9749,1,07-05-15,wine,19.61,0.54
12818,18.611047,12.9659,10.3815,10.4593,11.6881,12.3828,14.1274,7.18636,7.77498,25.7585,58.9683,1,07-05-15,wine,19.61,0.54
12819,18.611325,12.9648,10.3816,10.4592,11.6878,12.3833,14.1269,7.18663,7.77478,25.7596,58.9623,1,07-05-15,wine,19.61,0.54


# Building the final splitting function

In [170]:
def split_series_byID(n_ids, train_perc, joint_df):
    # Sampling test indices
    n_train = math.floor(train_perc * n_ids)
    train_indices = np.random.choice(np.arange(len_indices), size=n_train, replace=False)

    # Selecting train/test examples
    bool_list = []
    for id in joint_df.id:
        if id in train_indices:
            bool_list.append(True)
        else:
            bool_list.append(False)

    # We will use the fact it is a numpy array later
    bool_list = np.asarray(bool_list)

    df_train = joint_df[bool_list]
    # The fact that bool_list is a np array allows us to just choose its complementary
    df_test = joint_df[~bool_list]

    return df_train, df_test

In [171]:
df_meta = pd.read_csv('../datasets/raw/HT_Sensor_metadata.dat', delimiter='\t+')
df_meta.shape

(100, 5)

In [172]:
df_db = pd.read_csv('../datasets/raw/HT_Sensor_dataset.dat', delimiter='\s+')
df_db.shape

(928991, 12)

In [173]:
df_db.set_index('id', inplace=True)
df_db = df_db.join(df_meta, how='inner')
df_db['time'] += df_db['t0']
df_db.set_index(np.arange(df_db.shape[0]), inplace=True)
df_db.head()

,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity,id,date,class,t0,dt
0,12.490250,12.8621,10.3683,10.4383,11.6699,13.4931,13.3423,8.04169,8.73901,26.2257,59.0528,0,07-04-15,banana,13.49,1.64
1,12.490528,12.8617,10.3682,10.4375,11.6697,13.4927,13.3412,8.04133,8.73908,26.2308,59.0299,0,07-04-15,banana,13.49,1.64
2,12.490806,12.8607,10.3686,10.4370,11.6696,13.4924,13.3405,8.04101,8.73915,26.2365,59.0093,0,07-04-15,banana,13.49,1.64
3,12.491084,12.8602,10.3686,10.4370,11.6697,13.4921,13.3398,8.04086,8.73936,26.2416,58.9905,0,07-04-15,banana,13.49,1.64
4,12.491373,12.8595,10.3688,10.4374,11.6699,13.4919,13.3390,8.04087,8.73986,26.2462,58.9736,0,07-04-15,banana,13.49,1.64


In [174]:
n_ids = 100
train_perc = 0.75
df_train, df_test = split_series_byID(n_ids, train_perc, df_db)

In [175]:
print(df_train.shape)
df_train.head()

(710889, 16)


,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity,id,date,class,t0,dt
0,12.490250,12.8621,10.3683,10.4383,11.6699,13.4931,13.3423,8.04169,8.73901,26.2257,59.0528,0,07-04-15,banana,13.49,1.64
1,12.490528,12.8617,10.3682,10.4375,11.6697,13.4927,13.3412,8.04133,8.73908,26.2308,59.0299,0,07-04-15,banana,13.49,1.64
2,12.490806,12.8607,10.3686,10.4370,11.6696,13.4924,13.3405,8.04101,8.73915,26.2365,59.0093,0,07-04-15,banana,13.49,1.64
3,12.491084,12.8602,10.3686,10.4370,11.6697,13.4921,13.3398,8.04086,8.73936,26.2416,58.9905,0,07-04-15,banana,13.49,1.64
4,12.491373,12.8595,10.3688,10.4374,11.6699,13.4919,13.3390,8.04087,8.73986,26.2462,58.9736,0,07-04-15,banana,13.49,1.64


In [176]:
print(df_test.shape)
df_test.head()

(218102, 16)


,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity,id,date,class,t0,dt
31227,5.490119,11.5751,8.32428,8.43391,9.40930,11.1259,11.9891,7.89410,5.69867,26.3565,65.0541,3,07-09-15,banana,6.49,0.72
31228,5.490397,11.5752,8.32434,8.43402,9.40951,11.1259,11.9892,7.89354,5.69871,26.3559,65.0520,3,07-09-15,banana,6.49,0.72
31229,5.490675,11.5744,8.32410,8.43397,9.40937,11.1258,11.9895,7.89248,5.69865,26.3553,65.0500,3,07-09-15,banana,6.49,0.72
31230,5.490953,11.5737,8.32389,8.43393,9.40957,11.1258,11.9897,7.89151,5.69859,26.3548,65.0483,3,07-09-15,banana,6.49,0.72
31231,5.491239,11.5737,8.32413,8.43404,9.40992,11.1260,11.9900,7.89078,5.69853,26.3544,65.0467,3,07-09-15,banana,6.49,0.72


In [162]:
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
xtrain, ytrain = df_train[features].values, df_train['class'].values
xtest, ytest = df_test[features].values, df_test['class'].values

In [168]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(703127, 11)
(703127,)
(225864, 11)
(225864,)


In [165]:
C = 1e-2
clf_svm = SVC(C=C)

In [166]:
start_t = time.time()

clf_svm.fit(xtrain, ytrain)

end_t = time.time()

print('==> Tiempo transcurrido de entrenamiento (horas):', (end_t-start_t)/(60*60))

==> Tiempo transcurrido de entrenamiento (horas): 2.160339769191212


In [167]:
clf_svm.score(xtest, ytest)

0.3754338894201821

In [177]:
clf_lr = LogisticRegressionCV()
###
# Default: k-fold: 5-fold cross validation
#          Cs: A grid of Cs values are chosen in a logarithmic scale between 1e-4 and 1e4
#          penalty: l2
#          solver: lbfgs
#          tol: 1e-4
#          max_iter: 100
###

In [178]:
clf_lr.fit(xtrain, ytrain)

LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

In [179]:
clf_lr.score(xtest, ytest)

0.552509474728155